In [ ]:
!pip install transformers

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn.functional as F

# Load model & tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

# Your input text
text = "<|endoftext|>"
inputs = tokenizer(text, return_tensors="pt")

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # shape: [batch, seq_len, vocab_size]

# Get the logits for the last token in the sequence
last_token_logits = logits[0, -1, :]  # shape: [vocab_size]

# Convert to probabilities
probs = F.softmax(last_token_logits, dim=-1)

# Pair tokens with their probabilities
vocab_size = probs.shape[0]
token_probs = [
    (tokenizer.decode([i]), float(probs[i]))
    for i in range(vocab_size)
]

# Sort by probability (ascending)
token_probs.sort(key=lambda x: x[1], reverse=True)
print(len(token_probs))
token_probs[:100]

# Show top 10
# for token, prob in token_probs[:50]:
#     print(f"{repr(token)}: {prob:.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

50257


[('\n', 0.062299374490976334),
 ('The', 0.037699662148952484),
 ('"', 0.024113107472658157),
 ('A', 0.019402865320444107),
 ('I', 0.018320856615900993),
 ('In', 0.01157824881374836),
 ('.', 0.011301930993795395),
 ('It', 0.009115578606724739),
 ('S', 0.008656041696667671),
 ('This', 0.00776562187820673),
 ('B', 0.007691503968089819),
 ('-', 0.007649777922779322),
 ('C', 0.007192224729806185),
 ('We', 0.007012872025370598),
 ('1', 0.006710261572152376),
 ('T', 0.006652228534221649),
 ("'", 0.0059997085481882095),
 ('P', 0.005745659116655588),
 ('(', 0.005745155271142721),
 ('G', 0.005513082258403301),
 ('D', 0.005227970890700817),
 ('H', 0.005113239400088787),
 ('M', 0.004925859160721302),
 ('You', 0.004868842661380768),
 ('F', 0.004825059324502945),
 ('K', 0.004667073022574186),
 ('If', 0.004618915729224682),
 ('2', 0.004618281498551369),
 (':', 0.004275450948625803),
 ('R', 0.004266750533133745),
 ('L', 0.0040709576569497585),
 ('U', 0.003949577454477549),
 ('For', 0.00391255319118499